# Глава 9: Создание игр с помощью искусственного интеллекта

## Установка библиотеки easyAI

In [ ]:
!pip install easyAI

## Создание робота для игры Last Coin Standing ("Последняя монета")


In [ ]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax


class LastCoinStanding(TwoPlayerGame):
    def __init__(self, players):
        self.players = players
        
        # Первый играющий
        self.current_player = 1
        
        # Общее количество монет
        self.num_coins = 30
        
        # Максимальное кол-во монет
        self.max_coins = 4
    
    # ОВозможные ходы
    def possible_moves(self):
        return [str(x) for x in range(1, min(self.max_coins + 1, self.num_coins + 1))]
    
    # Удаление монет
    def make_move(self, move):
        self.num_coins -= int(move)
    
    # Проверка выигрыша
    def lose(self):
        return self.num_coins <= 0
    
    # Прекращение из-за победы
    def is_over(self):
        return self.lose()
    
    # Вычисление оценки
    def scoring(self):
        return 100 if self.lose() else 0
    
    # Отображение количества монет, оставшихся в куче
    def show(self):
        print(f'{self.num_coins} монет осталось в куче')


if __name__ == "__main__":
    print("=== Игра 'Последняя монета' ===")
    print("Правила: В куче 30 монет. За один ход можно взять от 1 до 4 монет.")
    print("Кто возьмет последнюю монету - ПРОИГРЫВАЕТ!")
    print("Вы играете против ИИ. Удачи!")
    print()
    
    # Создаем ИИ с алгоритмом Negamax (глубина поиска = 10)
    ai_algo = Negamax(10)
    
    # Начало игры: Игрок 1 = ИИ, Игрок 2 = Человек
    game = LastCoinStanding([AI_Player(ai_algo), Human_Player()])
    game.play()
    
    print()
    if game.lose():
        winner = 2 if game.current_player == 1 else 1
        if winner == 2:
            print("🎉 Поздравляем! Вы выиграли!")
        else:
            print("💻 ИИ победил! Попробуйте еще раз.")

## Создание робота для игры  Tic-Tac-Toe ("Крестики-нолики")


In [ ]:
from easyAI import TwoPlayerGame, AI_Player, Negamax, Human_Player


class GameController(TwoPlayerGame):
    def __init__(self, players):
        self.players = players
        
        # Определение того, кто начинает игру
        self.current_player = 1
        
        # Определение доски (9 пустых клеток)
        self.board = [0] * 9
    
    # Определение возможных ходов
    def possible_moves(self):
        return [str(a + 1) for a, b in enumerate(self.board) if b == 0]
    
    # Выполнение хода
    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player
    
    # Заполнил ли противник три клетки подряд?
    def loss_condition(self):
        possible_combinations = [
            [1, 2, 3], [4, 5, 6], [7, 8, 9],  # Горизонтали
            [1, 4, 7], [2, 5, 8], [3, 6, 9],  # Вертикали
            [1, 5, 9], [3, 5, 7]              # Диагонали
        ]
        return any([all([(self.board[i - 1] == self.opponent_index) 
                        for i in combination]) 
                   for combination in possible_combinations])
    
    # Проверка того, закончилась ли игра
    def is_over(self):
        return (len(self.possible_moves()) == 0) or self.loss_condition()
    
    # Отображение текущей позиции
    def show(self):
        symbols = ['.', 'O', 'X']
        print('\n' + '\n'.join([
            ' '.join([symbols[self.board[3 * j + i]] 
                     for i in range(3)]) 
            for j in range(3)
        ]))
    
    # Вычисление оценки
    def scoring(self):
        return -100 if self.loss_condition() else 0


if __name__ == "__main__":
    print("=== Игра 'Крестики-нолики' ===")
    print("Правила: Вы играете за 'O', компьютер за 'X'")
    print("Заполните три клетки подряд по горизонтали, вертикали или диагонали")
    print("Нумерация клеток:")
    print("1 2 3")
    print("4 5 6")
    print("7 8 9")
    print()
    
    # Определение алгоритма (глубина поиска = 7)
    algorithm = Negamax(7)
    
    # Запуск игры (Человек играет первым)
    game = GameController([Human_Player(), AI_Player(algorithm)])
    game.play()
    
    # Вывод результата
    print()
    if game.loss_condition():
        winner = 2 if game.current_player == 1 else 1
        if winner == 1:
            print("🎉 Поздравляем! Вы выиграли!")
        else:
            print("💻 ИИ победил! Попробуйте еще раз.")
    else:
        print("🤝 Ничья! Хорошая игра!")

## Создание двух роботов, играющих между собой в игру Connect Four ("Четыре в ряд")


In [ ]:
import numpy as np
import time
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax
try:
    from easyAI import SSS
except ImportError:
    SSS = None  # SSS может отсутствовать в некоторых версиях


class GameController(TwoPlayerGame):
    def __init__(self, players, board=None):
        # Определение игроков
        self.players = players
        
        # Определение конфигурации доски (6 рядов x 7 столбцов)
        self.board = board if (board is not None) else (
            np.array([[0 for i in range(7)] for j in range(6)]))
        
        # Определение того, кто начинает игру
        self.current_player = 1
        
        # Определение позиций для проверки выигрыша
        self.pos_dir = np.array(
            [[[i, 0], [0, 1]] for i in range(6)] +     # Горизонтали
            [[[0, i], [1, 0]] for i in range(7)] +     # Вертикали
            [[[i, 0], [1, 1]] for i in range(1, 3)] +  # Диагонали ↘ (нижний левый)
            [[[0, i], [1, 1]] for i in range(4)] +     # Диагонали ↘ (верхний левый)
            [[[i, 6], [1, -1]] for i in range(1, 3)] + # Диагонали ↙ (нижний правый)
            [[[0, i], [1, -1]] for i in range(3, 7)]   # Диагонали ↙ (верхний правый)
        )
    
    # Определение возможных ходов
    def possible_moves(self):
        return [str(i) for i in range(7) if (self.board[:, i].min() == 0)]
    
    # Определение хода, который следует сделать
    def make_move(self, column):
        column = int(column)
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player
        
        # Показываем доску после каждого хода
        print(f"\n{'='*30}")
        print(f"Игрок {self.current_player} ({'O' if self.current_player == 1 else 'X'}) ходит в колонку {column}")
        print('='*30)
        self.show()
        time.sleep(1.5)  # Пауза для наблюдения
    
    # Отображение текущего состояния
    def show(self):
        print('\n  0 1 2 3 4 5 6')
        print('  ' + 13 * '-')
        for j in range(6):
            row = ' '.join(['.', 'O', 'X'][self.board[5 - j][i]]
                          for i in range(7))
            print(f'{5-j} {row}')
        print()
    
    # Определение условий проигрыша
    def loss_condition(self):
        for pos, direction in self.pos_dir:
            streak = 0
            current_pos = pos.copy()
            
            while (0 <= current_pos[0] <= 5) and (0 <= current_pos[1] <= 6):
                if self.board[current_pos[0], current_pos[1]] == self.opponent_index:
                    streak += 1
                    if streak == 4:
                        return True
                else:
                    streak = 0
                
                current_pos = current_pos + direction
        
        return False
    
    # Проверка того, закончилась ли игра
    def is_over(self):
        return (self.board.min() > 0) or self.loss_condition()
    
    # Вычисление оценки
    def scoring(self):
        return -100 if self.loss_condition() else 0


if __name__ == '__main__':
    print("=== Игра 'Четыре в ряд' (Connect Four) ===")
    print("Правила: Выстройте 4 фишки подряд по горизонтали, вертикали или диагонали")
    print("Доска: 6 рядов x 7 столбцов")
    print("Столбцы пронумерованы от 0 до 6")
    print()
    
    # Определение используемых алгоритмов
    algo_neg = Negamax(5)
    
    # Проверяем доступность SSS
    if SSS is not None:
        algo_sss = SSS(5)
        print("🤖 Игрок 1 (O): Negamax")
        print("🤖 Игрок 2 (X): SSS*")
        print("\nДва ИИ играют друг против друга...")
        print("Наблюдайте за процессом игры!\n")
        
        # Начало игры (ИИ против ИИ)
        game = GameController([AI_Player(algo_neg), AI_Player(algo_sss)])
    else:
        print("⚠️  Алгоритм SSS недоступен в вашей версии easyAI")
        print("🤖 Игрок 1 (O): Negamax (глубина 5)")
        print("🤖 Игрок 2 (X): Negamax (глубина 4)")
        print("\nДва ИИ с Negamax играют друг против друга...")
        print("Наблюдайте за процессом игры!\n")
        
        # Начало игры (Negamax против Negamax)
        algo_neg2 = Negamax(4)  # Немного слабее для разнообразия
        game = GameController([AI_Player(algo_neg), AI_Player(algo_neg2)])
    
    # Показываем начальное состояние
    print("Начальное состояние доски:")
    game.show()
    time.sleep(2)
    
    # Начинаем игру
    game.play()
    
    # Вывод результата
    print("\n" + "="*40)
    print("ИГРА ОКОНЧЕНА!")
    print("="*40)
    if game.loss_condition():
        winner = 2 if game.current_player == 1 else 1
        symbol = 'O' if winner == 1 else 'X'
        print(f'🏆 Игрок {winner} ({symbol}) победил!')
    else:
        print("🤝 Ничья!")
    
    print("\n💡 Совет: Чтобы играть самому против ИИ, замените один AI_Player на Human_Player")
    print("   Например: GameController([Human_Player(), AI_Player(algo_neg)])")

## Создание двух роботов, играющих между собой в игру  Hexapawn ("Шесть пешек") 


In [ ]:
import time
from easyAI import TwoPlayerGame, AI_Player, Human_Player, Negamax


class GameController(TwoPlayerGame):
    def __init__(self, players, size=(4, 4)):
        self.size = size
        num_pawns, len_board = size
        
        # Создаем список позиций пешек для обоих игроков
        p = [[(i, j) for j in range(len_board)]
             for i in [0, num_pawns - 1]]
        
        # Определим направления, цели и пешки для каждого игрока
        for i, d, goal, pawns in [(0, 1, num_pawns - 1, p[0]),
                                   (1, -1, 0, p[1])]:
            players[i].direction = d
            players[i].goal_line = goal
            players[i].pawns = pawns
        
        # Определение игроков
        self.players = players
        
        # Определение того, кто начинает игру
        self.current_player = 1
        
        # Определение рабочего алфавита
        self.alphabets = 'ABCDEFGHIJ'
        
        # Преобразование B4 в (1, 3)
        self.to_tuple = lambda s: (self.alphabets.index(s[0]),
                                   int(s[1:]) - 1)
        
        # Преобразование (1, 3) в B4
        self.to_string = lambda move: ' '.join([self.alphabets[
            move[i][0]] + str(move[i][1] + 1)
            for i in (0, 1)])
    
    # Определение возможных ходов
    def possible_moves(self):
        moves = []
        opponent_pawns = self.opponent.pawns
        d = self.player.direction
        
        # Если в намеченной позиции отсутствует пешка противника,
        # то переход в эту позицию является допустимым
        for i, j in self.player.pawns:
            # Проверяем границы доски
            if 0 <= i + d < self.size[0]:
                # Ход вперед (если клетка свободна)
                if (i + d, j) not in opponent_pawns and \
                   (i + d, j) not in self.player.pawns:
                    moves.append(((i, j), (i + d, j)))
                
                # Взятие по диагонали вправо
                if j + 1 < self.size[1] and (i + d, j + 1) in opponent_pawns:
                    moves.append(((i, j), (i + d, j + 1)))
                
                # Взятие по диагонали влево
                if j - 1 >= 0 and (i + d, j - 1) in opponent_pawns:
                    moves.append(((i, j), (i + d, j - 1)))
        
        return [self.to_string(move) for move in moves]
    
    # Определение хода
    def make_move(self, move):
        move_tuple = list(map(self.to_tuple, move.split(' ')))
        ind = self.player.pawns.index(move_tuple[0])
        self.player.pawns[ind] = move_tuple[1]
        
        # Если взяли пешку противника
        if move_tuple[1] in self.opponent.pawns:
            self.opponent.pawns.remove(move_tuple[1])
        
        # Показываем доску после каждого хода
        print(f"\n{'='*40}")
        print(f"Игрок {self.current_player} ходит: {move}")
        print('='*40)
        self.show()
        time.sleep(1.5)
    
    # Определение условий проигрыша
    def loss_condition(self):
        return (any([i == self.opponent.goal_line
                    for i, j in self.opponent.pawns])
                or (len(self.possible_moves()) == 0))
    
    # Проверка того, закончилась ли игра
    def is_over(self):
        return self.loss_condition()
    
    # Отображение текущего состояния
    def show(self):
        def symbol(pos):
            if pos in self.players[0].pawns:
                return '1'
            elif pos in self.players[1].pawns:
                return '2'
            else:
                return '.'
        
        print()
        print('  ' + ' '.join(self.alphabets[:self.size[1]]))
        for i in range(self.size[0]):
            row = ' '.join([symbol((i, j)) for j in range(self.size[1])])
            print(f'{i+1} {row}')
        print()


if __name__ == '__main__':
    print("=== Игра Hexapawn (Шахматы с пешками) ===")
    print("Правила:")
    print("- Доска 4x4, у каждого игрока по 4 пешки")
    print("- Пешки ходят вперед на 1 клетку или берут по диагонали")
    print("- Побеждает тот, кто первым доведет пешку до противоположного края")
    print("- Или тот, у кого противник не может сделать ход")
    print()
    print("🤖 Игрок 1 (внизу): пешки '1', идет вверх")
    print("🤖 Игрок 2 (вверху): пешки '2', идет вниз")
    print()
    
    # Вычисление оценки
    scoring = lambda game: -100 if game.loss_condition() else 0
    
    # Определение алгоритма (NegaMax с глубиной 8 для скорости)
    algorithm = Negamax(8, scoring)
    
    print("Начальное состояние доски:")
    game = GameController([AI_Player(algorithm),
                          AI_Player(algorithm)])
    game.show()
    time.sleep(2)
    
    # Запуск игры
    print("Начинаем игру! Два ИИ играют друг против друга...\n")
    game.play()
    
    # Вывод результата
    print("\n" + "="*40)
    print("ИГРА ОКОНЧЕНА!")
    print("="*40)
    winner = 2 if game.current_player == 1 else 1
    print(f'🏆 Игрок {winner} побеждает после {game.nmove} ходов!')
    print()
    print("💡 Совет: Чтобы играть самому, замените один AI_Player на Human_Player")
    print("   Например: GameController([Human_Player(), AI_Player(algorithm)])")